# Product Insight Analyst Case Study #

We have two similar systems in Xero, full BAS and simpler BAS, for users to fulfill their compliance needs with the ATO. Attached are some extracts of times when each of the reports were being run by our users (simplebas.csv and fullbas.csv), and some metadata about the organisations (orgcard.csv).



Limit your analysis to active, paying organisations (orgs).

In this scenario, you are looking to present insights to the product manager of BAS at Xero. They would like to see:
1. an overview of how and when users interact with each system;
2. a comparison in terms of usage of each system;
3. a view of seasonality that could inform the need to increase compute capacity;
4. your recommendation on what the best time of month would be to have a two hour
outage window for each system.

Here are some key data points to include in your data story:
- What month saw the most report runs?
- How many orgs ran both reports?
- How many users have used either report?
- How many users have run reports for multiple organisations?
- Which pricing plans are the most popular amongst organisations using BAS?

### Note: 
Import variable: time stamps, userid, orgid, productopion, payingflag, runtime.

### Main Idea:

We can build a dashboard that performs time series analyses on FullBAS, and SimpleBAS data. \
First, we can create a plot of general system usage (user/org agnostic) between Full and Simple, 2 overlaying histograms. \
Then, decompose the above to filter for organizations. (we might even be able to do some kind of orgnisation classification here base on usage frequency) \
Hone into FullBAS, weigh the histograms by runtime to identify bottle necks/choking period to determine the need for compute capacity, and spot offtime for best outage window. 



We can implement some basic filters:
1. time
2. paying flags. 
3. productoption
4. orgstatus

## Assumptions

1. All users in simple BAS belong to Orgs that are paying Orgs

In [2]:
import pandas as pd
from datetime import date, datetime,  timedelta
import copy
import plotly.graph_objects as go


from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
import dash_daq as daq

from dash.dependencies import Input, Output
import plotly.express as px

pd.options.plotting.backend = "plotly"

In [73]:
orgdetails = pd.read_csv('data/orgcard.csv')
orgdetails['organisationid'] =orgdetails['organisationid'].str.lower()
orgdetails.rename(columns={'createddateutc':'datetime'}, inplace=True)
orgdetails['datetime'] = orgdetails['datetime'].astype('datetime64').dt.tz_localize('GMT').dt.tz_convert('Australia/Sydney')



fullbas = pd.read_csv('data/fullbas.csv')
fullbas['datetime'] = fullbas['datestring']+' '+fullbas['timestring']
fullbas['datetime'] = fullbas['datetime'].astype('datetime64').dt.tz_localize('GMT').dt.tz_convert('Australia/Sydney')

simplebas = pd.read_csv('data/simplebas.csv')
simplebas['datetime'] = simplebas['datetime'].astype('datetime64[s]').dt.tz_localize('GMT').dt.tz_convert('Australia/Sydney')

orgdetails

,organisationid,shardid,shortcode,organisationstatus,datetime,signupdateutc,productoption,deleteddateutc,saleschannel,channeltype,...,salestaxperiod,salestaxtype,incometaxbasis,provisionaltaxbasis,managedbypracticeflag,payingflag,marketcode,nonpracticestaffusers,practicestaffusers,trialflag
0,cdeab7fb-e43f-4344-904d-3137fed5aac7,87,!Qddkq,Active,2015-10-14 16:32:10.737000+11:00,2020-02-13 00:00:00,Standard,NaN,Partner,Accountant,...,Australian Quarterly (Option 1),Australian GST,None,None,1,1,AU,2,6,0
1,726a4578-00d6-4357-bfe7-85d1a656eebf,121,!DhWsK,Active,2018-08-06 10:52:15.520000+10:00,2018-08-06 00:00:00,Standard,NaN,Partner,Accountant,...,Australian Quarterly (Option 1),Australian GST,None,None,1,1,AU,0,15,0
2,5cdf4968-430d-4c55-a52e-845cd144b786,116,!QPRI6,Active,2014-11-03 10:26:12.553000+11:00,2014-11-03 00:00:00,Standard,NaN,Partner,Accountant,...,Australian Quarterly (Option 1),Australian GST,None,None,1,1,AU,1,18,0
3,8f483757-ace6-4a36-8805-10481bccac42,116,!4VKHh,Active,2014-10-22 12:55:59.020000+11:00,2014-10-22 00:00:00,Standard,NaN,Partner,Bookkeeper,...,Australian Quarterly (Option 1),Australian GST,None,None,1,1,AU,1,3,0
4,195aa44c-dd6d-42a9-914a-552086303b2b,133,!hWiKs,Active,2015-12-07 11:09:08+11:00,2017-03-15 00:00:00,Premium 5,NaN,Partner,Bookkeeper,...,Australian Monthly,Australian GST,Australian Quarterly Installment (Option 1),None,1,1,AU,1,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1466,c977d42c-7940-4d14-a630-853530cc16fa,126,!3ixKu,Active,2015-04-01 18:21:30+11:00,2015-04-29 00:00:00,Premium 5,NaN,Partner,Accountant,...,Australian Quarterly (Option 1),Australian GST,None,None,1,1,AU,1,8,0
1467,b0762bcf-c8b9-468f-af09-2ec43e996fe1,115,!Qb7nf,Active,2018-01-12 14:32:37+11:00,2018-01-13 00:00:00,Standard,NaN,Partner,Accountant,...,Australian Quarterly (Option 1),Australian GST,Australian Quarterly Installment (Option 1),None,1,1,AU,1,2,0
1468,e8f399d0-e946-4410-9679-e3b92bae956d,115,!y4ckL,Active,2018-08-30 09:52:44+10:00,2018-08-31 00:00:00,Standard,NaN,Online,Unknown,...,Australian Quarterly (Option 1),Australian GST,None,None,0,1,AU,2,0,0
1469,9b29eb00-eb57-4d1c-9ead-f3c294fcf4cf,156,!A86SH,Active,2018-06-25 14:43:43+10:00,2018-11-20 00:00:00,Standard,NaN,Partner,Accountant,...,Australian Monthly,Australian GST,Australian Income Times (Option 2),None,1,1,AU,3,3,0


In [ ]:
simpledf=simplebas.groupby([simplebas['datetime'].dt.date]).size().reset_index(name='count')
fulldf = fullbas.groupby([fullbas['datetime'].dt.date]).size().reset_index(name='count')


In [ ]:
def filter_by_time(df, start_date, end_date):

    dff = df[
        (df["datetime"].dt.date > datetime.strptime(start_date, "%Y-%m-%d").date())
        & (df["datetime"].dt.date < datetime.strptime(end_date, "%Y-%m-%d").date())
    ]
    return dff

def filter_by_payingflag(df, paying_status):
    return df[df['payingflag'].isin(list(map(int, paying_status)))]

def filter_by_status(df, status):
    return df[df['organisationstatus'].isin(list(map(int, status)))]



In [ ]:
simpledf=simplebas.groupby([simplebas['datetime'].dt.date]).size().reset_index(name='count')
fulldf = fullbas.groupby([fullbas['datetime'].dt.date]).size().reset_index(name='count')

In [ ]:
layout = dict(
    autosize=True,
    automargin=True,
    margin=dict(l=30, r=30, b=20, t=40),
    hovermode="closest",
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#F9F9F9",
    legend=dict(font=dict(size=10), orientation="h"),
)

app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.DatePickerRange(
                    id="date_picker",
                    start_date = '2016-10-01',
                    end_date=date.today(),
                    display_format='MMM Do, YY',
                    start_date_placeholder_text='MMM Do, YY'
                ),
    dcc.Checklist(
                    id="payment_status",
                    options={
                            '1': 'Paying',
                            '0': 'Non-Paying',
                    },
                    value=['1']
                    ),
     daq.BooleanSwitch(
                        id='dailyweeklytoggle',
                        on=True,
                        label="Daily/Weekly",
                        labelPosition="bottom",
                        style= {'width': 'fit-content'}
                        ),
    dcc.Graph(id="usage_graph"),
])
@app.callback(
    Output("usage_graph", "figure"), 
    Input("date_picker", "start_date"),
    Input("date_picker", "end_date"),
    Input("payment_status", "value"),
    Input("dailyweeklytoggle","on")
    )
def update_usage_graph(start_date, end_date, payment_status, dailyweeklytoggle): 
    
    layout_aggregate = copy.deepcopy(layout)

    fullbas_w_org = pd.merge(fullbas, orgdetails, left_on='orgid', right_on='organisationid', how="inner")
    fullbas_w_org = filter_by_payingflag(fullbas_w_org,payment_status)

    # simpledf=simplebas.groupby([simplebas['datetime'].dt.date]).size().reset_index(name='count')
    # fulldf = fullbas_w_org.groupby([fullbas_w_org['datetime'].dt.date]).size().reset_index(name='count')
    if dailyweeklytoggle is False:
        simpledf = simplebas.groupby(pd.Grouper(freq='D', key='datetime')).size().reset_index(name='count')
        fulldf = fullbas_w_org.groupby(pd.Grouper(freq='D', key='datetime')).size().reset_index(name='count')
    else:

        simpledf = simplebas.groupby(pd.Grouper(freq='W', key='datetime')).size().reset_index(name='count')
        fulldf = fullbas_w_org.groupby(pd.Grouper(freq='W', key='datetime')).size().reset_index(name='count')

        simpledf['count'] = simpledf.rolling(window = 7).mean()
        fulldf['count'] = fulldf.rolling(window = 7).mean()

    simpledff = filter_by_time(simpledf, start_date, end_date)
    fulldff = filter_by_time(fulldf, start_date, end_date)
    
    data = [
            dict(
                type="line",
                name="Full",
                x=fulldff['datetime'],
                y=fulldff['count'],
                line=dict(shape="spline", smoothing=2, width=1, ),
            ),
            dict(
                type="line",
                name="Simple",
                x=simpledff['datetime'],
                y=simpledff['count'],
                line=dict(shape="spline", smoothing=2, width=1),
            ),
            ]
    figure = dict(data=data, layout=layout_aggregate)
    return figure
app.run_server(mode='inline', port=8050, debug=True)

In [ ]:
def isOneToOne(df, col1, col2):
    first = df.drop_duplicates([col1, col2]).groupby(col1)[col2].count().max()
    second = df.drop_duplicates([col1, col2]).groupby(col2)[col1].count().max()
    return first + second == 2


In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
df = fullbas.groupby(pd.Grouper(freq='D', key='datetime')).size().reset_index(name='count')
# df2 = fullbas.groupby(pd.Grouper(freq='W', key='datetime')).size().reset_index(name='count')


rolling_mean = df.rolling(window = 30).mean()
rolling_std = df.rolling(window = 30).std()

df['rolling_mean'] = rolling_mean
df['rolling_std'] = rolling_std
figure  = go.Figure()

figure.add_trace(
    go.Scatter(
        x = df['datetime'],
        y = df['count'],
        name = 'Daily'
    )
)

figure.add_trace(
    go.Scatter(
        x = df['datetime'],
        y = df['rolling_mean'],
        name = 'Rolling Mean'
    )
)

figure.add_trace(
    go.Scatter(
        x = df['datetime'],
        y = df['rolling_std'],
        name = 'Rolling Std'
    )
)
figure.show()


In [ ]:
import numpy as np
import scipy.signal


L = df['rolling_mean'].dropna()
# L = np.round(L, 1)
# Remove DC component
L -= np.mean(L)
# Window signal
L *= scipy.signal.windows.hann(len(L))

fft = np.fft.rfft(L, norm="forward")
freq = np.fft.fftfreq(len(L), d=1)


figure = go.Figure()
figure.add_trace(
    go.Scatter(x= 1/freq, y = abs(fft))
    )
figure.show()



In [38]:
# simpledf_monthly=simplebas
simpledf_monthly = list(simplebas.groupby(simplebas['datetime'].dt.year)['datetime'])
fulldf_monthly = list(fullbas.groupby(fullbas['datetime'].dt.day)['datetime'])

figure = go.Figure()

for index, item, in enumerate(simpledf_monthly):
    
    daily = item[1].groupby(item[1].dt.month).count().reset_index(name='count')
    figure.add_trace(
    go.Scatter(x=daily['datetime'], y = daily['count'], name = 'Day %s' % (index+1))
    )
   
figure.show()


In [55]:
simpledf_monthly = (simplebas.groupby(fullbas['datetime'].dt.month)).size().reset_index(name='count')
fulldf_monthly = (fullbas.groupby(fullbas['datetime'].dt.month)).size().reset_index(name='count')

figure = go.Figure()


figure.add_trace(
go.Scatter(x=fulldf_monthly['datetime'], y = fulldf_monthly['count'], name = 'Full BAS' )
)
figure.add_trace(
go.Scatter(x=simpledf_monthly['datetime'], y = simpledf_monthly['count'], name = 'Simple BAS' )
)  
figure.show()



In [68]:
fullbas[['userid', 'orgid']]
simplebas['userid']

res = pd.merge(simplebas, fullbas, left_on='userid', right_on='userid', how="outer")

hist = res[['userid', 'orgid']].drop_duplicates().dropna().groupby('userid').size().reset_index(name='count')

figure = go.Figure(data =[ go.Histogram(x=hist['count'].values)])
figure.update_yaxes(type='log')
figure.show()

hist[hist['count']>10]

,userid,count
2003,6228e721-6c79-42ea-9a3d-30b9ae429716,68


In [54]:
fullbas[['userid', 'orgid']]
simplebas['userid']

inner_res = pd.merge(simplebas, fullbas, left_on='userid', right_on='userid', how="inner")
outer_res = pd.merge(simplebas, fullbas, left_on='userid', right_on='userid', how="outer")

print(len(fullbas))
print(len(simplebas))

#number of user that ran both BAS systems, also the number of org that ran both BAS system excluding different users runing for the sample org
inner_res[['userid', 'orgid']].drop_duplicates()

#number of user that ran either BAS systems
len(outer_res[['userid', 'orgid']].drop_duplicates())

239862
79405


6811

In [67]:
userorg = (res[['userid', 'orgid']].drop_duplicates().dropna().groupby('userid'))



In [83]:
product_pie = orgdetails['productoption'].groupby(orgdetails['productoption']).count().reset_index(name='count')
product_pie.loc[product_pie['count']<50, 'productoption'] = 'Other'

In [88]:
fig = go.Figure(data=[go.Pie(labels=product_pie['productoption'], values=product_pie['count'], hole=0.4)])
fig.show()